# Total Variation (TV) regularizáció PET rekonstrukcióhoz

A [pozitronemissziós tomográfia (PET)](https://hu.wikipedia.org/wiki/Pozitronemisszi%C3%B3s_tomogr%C3%A1fia) egy orvosi képalkotó eljárás, aminek során radioaktív nyomjelző anyagot juttatnak a páciensbe, amely nyomjelző felhalmozódik a tumorban és γ-sugárzás kibocsátásával felfedi annak helyét. A tomográf γ-fotonpárok becsapódását érzékeli, és ezekből a becsapódási adatokból kell szoftveresen rekonstruálni a páciens testét. A rekonstrukciót a [maximum-likelihood expectation-maximization (ML-EM)](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm) algoritmussal szokás végezni. Az ML-EM azonban sokszor zajos képet eredményez, ezért *regularizációra* van szükség. A [total variation (TV) regularizáció](https://en.wikipedia.org/wiki/Total_variation_denoising) egy olyan szűrési eljárás, amely a kimeneti kép *teljes varianciáját* (a kép gradiensének abszolút értékének az integrálját) csökkenti, mivel a nagy varianciát szinte mindig a zaj okozza. Ha a varianciát el tudjuk nyomni úgy, hogy az előálló kép a lehető legközelebb legyen a bemenethez, akkor lényegében *zajtalanítjuk* a bemeneti képet. TV regularizációt a PET rekonstrukcióban úgy tudunk alkalmazni, hogy az ML-EM által minimalizált célfüggvénybe beillesztjük a *„+ λ⋅TV(x)”* tagot, ahol a *TV(x)* az aktuális nyomjelzőeloszlás-becslés (*x*) teljes varianciája és λ a regularizáció erőssége. (Minél nagyobb λ értéke, annál erősebb a regularizáció hatása, viszont annál inkább jelenhet meg nem kívánt elmosás.) A λ ideális értéke az ML-EM rekonstrukció során iterációról iterációra változhat, így a fix értéken rögzítés nem ad optimális eredményt, a kézzel történő beállítása azonban nem kivitelezhető (nincs rá ember, idő, illetve módszer). A mai gyakorlaton egy enkóder-dekóder neurális hálózatot fogunk betanítani a TV regularizáció λ paraméterének finomhangolására az ML-EM rekonstrukcióhoz.

## Importok és inicializálás

In [ ]:
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, Activation, Concatenate, Lambda, Flatten
from keras.layers import MaxPooling2D, AveragePooling2D, UpSampling2D, Conv2DTranspose
from keras.layers import BatchNormalization, Dropout
from keras.utils.vis_utils import plot_model 
import keras.callbacks

import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from tqdm.notebook import tqdm
import numpy as np

np.random.seed(42)

## Adatok letöltése



In [ ]:
!wget http://cg.iit.bme.hu/~drnyu/dlvizinf/pet_lab_input_data.zip
!unzip pet_lab_input_data.zip

## Adatok betöltése

Adott a következő segédfüggvény az adatok betöltéséhez:


In [ ]:
def load_data(dir):
    data = {}
    data['xin'] = np.load(dir + "/xin.npy")
    data['gradient'] = np.load(dir + "/gradient.npy")
    data['numerator'] = np.load(dir + "/numerator.npy")
    data['denominator'] = np.load(dir + "/denominator.npy")
    data['reference'] = np.load(dir + "/reference.npy")
    data['global_tv'] = np.load(dir + "/global_tv.npy")
    # Remark: data arrays have a shape of (n_samples, height, width, channels)
    return data

Feladat:

* Inicializáljátok a *xin*, *gradient*, *numerator*, *denominator*, *reference* változókat a betöltött adatokból!
* Keverjétek meg az adatokat egy random permutáció alkalmazásával úgy, hogy a változók közötti összhang a keverés után is megmaradjon! (Tipp: a [numpy.random.permutation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.permutation.html) permutált tartományt ad vissza.)
* A gyorsabb tanítás érdekében most csak az első 50.000 mintával fogunk dolgozni. Dobjátok el a többit!

In [ ]:
data = load_data('./pet_lab_input_data')

perm = None

xin         = None
gradient    = None
numerator   = None
denominator = None
reference   = None

data_limit  = 50000

xin         = None
gradient    = None
numerator   = None
denominator = None
reference   = None

print('Data shape: (n_samples, height, width, channels)')
print(xin.shape)

n_lambda_samples = 100
lambda_samples_delta = 0.002
lambda_errors = np.zeros((xin.shape[0], n_lambda_samples))
global_lambda = np.zeros(xin.shape[0])
result = np.zeros((xin.shape[0], 1))

## Vizualizáció

Mielőtt nekilátunk a TV regularizáció megvalósításához, vizualizáljuk az adatokat!

Feladat:
* Jelenítsétek meg a *reference*,  *xin*, *gradient*, *numerator*, *denominator* változók *x*-edik mintáját képként egymás mellett az [add_subplot](https://matplotlib.org/3.5.0/api/figure_api.html?highlight=add_subplot#matplotlib.figure.Figure.add_subplot) segítségével! A kirajzoláshoz használjátok a *'hot'* színtérképet!
* Készítsetek interaktív "nézegetőt", ahol az *x* értékét egy csúszka segítségével lehet állítani! (Tipp: [Interact](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html))

In [ ]:
def plot_slice(x):
  imNum = 5
  fig = plt.figure(figsize=(18, 7), dpi= 80)















  plt.show()

In [ ]:
# interact(...)

## A TV regularizáció λ paraméterének optimalizálása

Az ML-EM iterációs séma a következőképpen néz ki TV regularizáció alkalmazása esetén:

$$
x_{n+1} = x_{n} * \frac{numerator}{denominator + \lambda * gradient}
$$

A TV regularizáció λ paraméterét úgy kell beállítani, hogy $x_{n+1}$ a lehető legközelebb legyen a referenciához (L2 távolság értelmében).

Feladat:
* Implementáljátok a *get_lambda_errors* függvényt, amely kiszámolja, hogy a különböző λ értékek mellett $x_{n+1}$-nek mekkora lesz az L2 hibája a referenciától! (Emlékeztető: *n_lambda_samples* adja meg, hogy hány darab mintát szeretnénk venni λ-ból, és *lambda_samples_delta* megadja a minták közötti távolságot. A mintavételt nullától kezdjük.)
* Iteráljatok végig minden bemeneti adatmintán (*0 … xin.shape[0]*) és mindegyik mintához számoljátok ki, illetve tároljátok el a *global_lambda* tömbbe a legkisebb L2 hibát eredményező λ értéket!


In [ ]:
def get_lambda_errors(xin, denominator, gradient, numerator, reference, sample_idx, n_lambda_samples, lambda_samples_delta):
    error = np.zeros(n_lambda_samples)
    
    



    
    return error

In [ ]:
# Remark: tqdm adds a progress bar to track the progress of the for loop
for i in tqdm(range(0, xin.shape[0])):
  pass


## Neurális hálózat felépítése és tanítása

Feladat:
* Implementáljátok a következő modellt:

![tv_nn_architecture.png](http://cg.iit.bme.hu/~drnyu/dlvizinf/tv_nn_architecture.png)

Minden konvolúciós és dekonvolúciós réteg
* kernelmérete 3,
* zero (*same*) paddinget használ,
* ReLU aktivációjú,
* Batch Normalization réteg követi.

A konvolúciós szűrők száma legyen rendre 16, 32, 64, míg a dekonvolúciós rétegeké 64, 32, 16! A konvolúciós rétegek között alkalmazzatok average pooling-ot, míg a dekonvolúciós rétegek között upsampling-ot! (Az utolsó konvolúciós, illetve dekonvolúciós réteg után nem kell méretváltoztatás!) A dekonvolúciós rétegek kimeneteihez konkatenáljátok a megfelelő konvolúciós réteg kimenetét (lásd az ábrán a vízszintes nyilak), és ez a konkatenációs réteg legyen a következő dekonvolúciós réteg bemenete!

A három Dense réteg 8-8 neuronból álljon, és ReLU aktivációt használjanak! Őket kövesse egy Flatten réteg, majd pedig az output layer, ami egy 1 neuronból álló Dense réteg sigmoid aktivációval!

* Tanítsátok be a hálózatot! (A kód ehhez már készen áll, futtassátok le és értelmezzétek!)

In [ ]:
def get_model(rows, cols):
    inputX = Input(shape=(cols, rows, 1), name='x')
    inputG = Input(shape=(cols, rows, 1), name='grad')
    inputNum = Input(shape=(cols, rows, 1), name='num')

    merge0 = Concatenate(axis=-1, name='concat_1')([inputX, inputG, inputNum])
    conv1 = # ...
    # ...
    output = # ...

    model = Model(inputs=[inputX, inputG, inputNum], outputs=[output])

    return model

In [ ]:
def train(model, xin, numerator, gradient, global_lambda, prefix):
    #plot_model(model, to_file=prefix + '_model.png')
    stop_cb = keras.callbacks.EarlyStopping\
        (monitor='val_loss', min_delta=0.01, patience=10, verbose=1, mode='auto')
    save_cb = keras.callbacks.ModelCheckpoint(prefix + '_weights.{epoch:02d}-{val_loss:.2f}.hdf5',
                                              save_weights_only=True, period=10)
    history = model.fit(x=[xin[:, :, :, :], gradient[:, :, :, :], numerator[:, :, :, :]],
                        y=global_lambda[:],
                        epochs=20,
                        batch_size=256,
                        validation_split=0.2,
                        shuffle=True,
                        callbacks=[save_cb])

    #model.save(prefix + '_tv_model.h5')
    #model.save_weights(prefix + '_tv_weights.h5')

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

In [ ]:

model = get_model(32, 32)
training_percent = 0.8

training_limit_idx = int(data_limit * training_percent)
print("Training limit: " + str(training_limit_idx))

model.summary()
model.compile(loss={'output' : 'mean_squared_logarithmic_error'}, optimizer='adam', metrics=['mse'])
#plot_model(model, "model.svg")

train(model, xin[0:training_limit_idx,], numerator[0:training_limit_idx,], gradient[0:training_limit_idx,], global_lambda[0:training_limit_idx,], "tv_model")


## Kiértékelés

A tanítás után kiértékeljük a modellt a többi adaton. Futtassátok le az alábbi kódblokkokat és vizsgáljátok meg a hálózat teljesítményét! Tudtok úgy javítani a hálózaton, hogy jobb eredményt érjen el?

In [ ]:
pred = model.predict(x=[xin[training_limit_idx:, :, :, :], gradient[training_limit_idx:, :, :, :], numerator[training_limit_idx:, :, :, :]])

In [ ]:
plt.plot(global_lambda[training_limit_idx:])
plt.plot(pred)
plt.legend(['Optimal', 'Predicted'])
plt.xlim(700, 1000)
plt.xlabel("Test number")
plt.ylabel("Lambda")
plt.show()

In [ ]:
errA = np.array(pred)
for i in range(pred.size):
    errA[i] = global_lambda[training_limit_idx + i] - pred[i]

np.histogram(errA)
plt.hist(errA, bins='auto')
plt.xlim(-0.2, 0.2)
plt.xlabel("Deviation")
plt.ylabel("Number of tests")
plt.show()